In [33]:
import pandas as pd

In [34]:
movies = pd.read_json("movies.json")
reviews = pd.read_json("reviews.json")
reviews.head()

,user_id,item_id,rating,timestamp
0,0,50,5,1997-12-04 15:55:49
1,0,172,5,1997-12-04 15:55:49
2,0,133,1,1997-12-04 15:55:49
3,196,242,3,1997-12-04 15:55:49
4,186,302,3,1998-04-04 19:22:22


In [35]:
reviews.drop(['timestamp'], inplace=True, axis=1)

In [36]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99710 entries, 0 to 99709
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   user_id  99710 non-null  int64
 1   item_id  99710 non-null  int64
 2   rating   99710 non-null  int64
dtypes: int64(3)
memory usage: 2.3 MB


In [37]:
mostReviewed = 3
topN = reviews.groupby('item_id').size()

In [38]:
topN = topN.sort_values(ascending=False).reset_index()[:mostReviewed]
topN

,item_id,0
0,50,583
1,258,507
2,100,506


In [39]:
top3DF = reviews[reviews['item_id'].isin(topN['item_id'])]

In [40]:
top3DF.head()

,user_id,item_id,rating
0,0,50,5
53,251,100,4
188,26,258,3
267,66,258,4
378,207,258,4


In [41]:
pivotDF = top3DF.pivot(index='user_id', columns='item_id', values='rating')

In [44]:
pivotDF = pivotDF.dropna(axis=0, how="all")

In [46]:
pivotDF = pivotDF.fillna(-1)

In [47]:
pivotDF.head()

item_id,50,100,258
user_id,,,
0,5.0,-1.0,-1.0
1,5.0,5.0,5.0
2,5.0,5.0,3.0
3,-1.0,-1.0,2.0
4,5.0,-1.0,5.0


In [48]:
from sklearn.cluster import MeanShift

In [49]:
ms = MeanShift(bandwidth=2)

In [51]:
ms = ms.fit(pivotDF)

In [52]:
ms.labels_

array([4, 0, 0, 2, 3, 1, 0, 0, 3, 4, 1, 6, 4, 0, 1, 3, 5, 5, 1, 2, 4, 0,
       3, 0, 6, 3, 0, 1, 0, 3, 1, 2, 2, 4, 2, 2, 1, 4, 0, 0, 1, 1, 2, 4,
       7, 5, 4, 6, 0, 0, 4, 3, 0, 1, 0, 4, 2, 0, 0, 0, 0, 3, 3, 0, 4, 1,
       1, 5, 6, 5, 6, 1, 0, 1, 5, 1, 4, 0, 2, 1, 1, 6, 4, 0, 0, 4, 1, 1,
       0, 2, 4, 3, 4, 0, 2, 5, 2, 1, 0, 2, 2, 2, 0, 5, 1, 3, 3, 6, 0, 3,
       1, 1, 4, 3, 2, 3, 3, 2, 0, 5, 1, 2, 2, 2, 6, 4, 5, 5, 2, 0, 2, 0,
       0, 2, 2, 4, 2, 1, 0, 2, 3, 5, 0, 1, 2, 1, 1, 4, 2, 6, 2, 2, 6, 3,
       2, 2, 2, 1, 1, 0, 0, 0, 2, 2, 6, 1, 3, 0, 3, 5, 1, 1, 6, 0, 6, 1,
       6, 3, 3, 6, 3, 0, 2, 3, 2, 2, 2, 6, 3, 4, 0, 1, 3, 1, 2, 5, 2, 2,
       1, 0, 2, 2, 2, 1, 2, 1, 4, 1, 1, 0, 1, 1, 5, 2, 1, 0, 3, 1, 0, 1,
       0, 0, 0, 5, 1, 3, 6, 1, 0, 2, 2, 7, 0, 5, 7, 5, 1, 0, 1, 3, 0, 4,
       0, 3, 0, 0, 2, 1, 1, 2, 2, 1, 2, 7, 0, 1, 0, 4, 1, 1, 0, 0, 1, 0,
       0, 4, 1, 5, 0, 2, 0, 0, 7, 0, 1, 2, 3, 3, 1, 0, 5, 5, 1, 1, 1, 3,
       0, 3, 1, 4, 7, 0, 0, 1, 5, 3, 5, 0, 2, 1, 4,

In [53]:
from sklearn.metrics import silhouette_score

In [54]:
ss = silhouette_score(pivotDF, ms.labels_)

In [55]:
ss

0.651624514271997

In [57]:
scores = {}
for i in range(2, 10):
    ms = MeanShift(bandwidth=i)
    ms = ms.fit(pivotDF)
    if(len(set(ms.labels_)) > 1):
        ss = silhouette_score(pivotDF, ms.labels_)
        scores[i] = ss

In [58]:
scores

{2: 0.651624514271997,
 3: 0.6819678355942147,
 4: 0.3667231373387644,
 5: 0.3374988034418743}

In [59]:
ms = MeanShift(bandwidth=3)
ms = ms.fit(pivotDF)

In [60]:
pivotDF['Cluster'] = ms.labels_

In [61]:
pivotDF.head()

item_id,50,100,258,Cluster
user_id,,,,
0,5.0,-1.0,-1.0,4
1,5.0,5.0,5.0,0
2,5.0,5.0,3.0,0
3,-1.0,-1.0,2.0,2
4,5.0,-1.0,5.0,3


In [62]:
# Let's take a look at user #1! What movie should they watch next?
user1 = reviews[reviews['user_id'] == 1]

In [63]:
user1

,user_id,item_id,rating
204,1,61,4
307,1,189,3
335,1,33,4
336,1,160,4
480,1,20,4
...,...,...,...
91778,1,28,4
92215,1,172,5
93743,1,122,3
96419,1,152,5


In [64]:
user2 = reviews[reviews['user_id'] == 2]

In [65]:
user2

,user_id,item_id,rating
702,2,292,4
926,2,251,5
1052,2,50,5
3420,2,314,1
5056,2,297,4
...,...,...,...
77680,2,288,3
85357,2,286,4
87936,2,275,5
95400,2,302,5


In [67]:
user1[user1['item_id'] == 302]

,user_id,item_id,rating


In [68]:
movies[movies['item_id'] == 302]

,item_id,title
301,302,L.A. Confidential (1997)
